# ERROR BETWEEN PREDICTED AND ACTUAL VALUES

## Webscraping part 

In [32]:
#This part is usually for the scraping part. I will be scraping according to the dates. This is because I want to deal with less data in the beginning and will try to implement the code on a larger scale.
# The date I have chosen is 30th April 2019.
import pandas as pd
import ftputil
import numpy as np
import os
import datetime
data = { 'Selection criteria': ['time range'], 'index_history.txt': [' ✔'], 'index_monthly.txt': [' ✔'],'index_latest.txt': [' ✔']}
pd.DataFrame(data=data)
output_directory = os.getcwd()
user = 'anathsharma'
password = '@n&1!MZx'
product_name = 'INSITU_BAL_NRT_OBSERVATIONS_013_032'  
host = 'nrt.cmems-du.eu' 
index_file = 'index_latest.txt'
date_format = "%Y-%m-%dT%H:%M:%SZ" 
ini = datetime.datetime.strptime('2019-04-30T00:00:00Z', date_format)
end = datetime.datetime.strptime('2019-04-30T23:59:59Z', date_format)
#We can scrape the data for as long as it is required and simultaneous data can be inserted in the dates.

In [2]:
# This part is for developing the intermediate between my laptop and the host server from where the data is scraped.
#I used the help of Marine 
#connect to CMEMS FTP
with ftputil.FTPHost(host, user, password) as ftp_host: 
    
    #open the index file to read
    with ftp_host.open("Core"+'/'+product_name+'/'+index_file, "r") as indexfile:
        
        #read the index file as a comma-separate-value file
        index = np.genfromtxt(indexfile, skip_header=6, unpack=False, delimiter=',', dtype=None, names=['catalog_id', 'file_name','geospatial_lat_min', 'geospatial_lat_max', 'geospatial_lon_min','geospatial_lon_max','time_coverage_start', 'time_coverage_end', 'provider', 'date_update', 'data_mode', 'parameters'])
               
        #loop over the lines/netCDFs and download the most sutable ones for you
        for netCDF in index:
            
            #getting ftplink, filepath and filename
            ftplink = netCDF['file_name'].decode('utf-8')
            filepath = '/'.join(ftplink.split('/')[3:len(ftplink.split('/'))])
            ncdf_file_name = ftplink[ftplink.rfind('/')+1:]
            
            #download netCDF if meeting selection criteria
            time_start = datetime.datetime.strptime(netCDF['time_coverage_start'].decode('utf-8'), date_format)
            time_end = datetime.datetime.strptime(netCDF['time_coverage_start'].decode('utf-8'), date_format)
            if time_start > ini  and time_end < end: 
                if ftp_host.path.isfile(filepath):
                    cwd = os.getcwd()
                    os.chdir(output_directory)
                    ftp_host.download(filepath, ncdf_file_name)  # remote, local
                    os.chdir(cwd)

c:\users\awshesh nath sharma\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `use_list_a_option` will default to `False` in ftputil 4.x.x
  
c:\users\awshesh nath sharma\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:8: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  


## Reading into csv format 

In [35]:
#For reading the actual insitu observations which is scraped from the site.
import netCDF4
from netCDF4 import Dataset
import numpy as np
file = Dataset("BO_LATEST_TS_FB_FinnMaid_20190430.nc",'r')

In [36]:
#finding the features of the file 
print(file)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    naming_authority: Copernicus
    area: Baltic Sea
    format_version: 1.2
    contact: cmems-service@smhi.se
    author: cmems-service
    cdm_data_type: Station
    summary: Oceanographic data from the Baltic Sea. Measured properties: the hydrographic conditions as currents, temperature and salinity.
    references: http://www.oceansites.org http://www.myocean.org
    data_assembly_center: Baltic INS TAC DU
    update_interval: daily
    citation: These data were collected and made freely available by the Copernicus project and the programs that contribute to it
    Conventions: CF-1.6 OceanSITES-Manual-1.2 Copernicus-InSituTAC-SRD-1.3 Copernicus-InSituTAC-ParametersList-3.1.0
    qc_manual: OceanSITES User's Manual v1.1
    distribution_statement: These data follow Copernicus standards; they are public and free of charge. User assumes all risk for use of data. User must display citati

In [37]:
# Extracting the Features of the File 
d=file.variables["LATITUDE"][:]
e=file.variables["LONGITUDE"][:]
#f=file.variables["POSITION_QC"][:]
g=file.variables["TEMP"][:]
h=file.variables["DEPH"][:]
po=file.variables["TIME"][:]


In [43]:
print(po)

[25321.09181713 25321.09206019 25321.09230324 ... 25321.37474537
 25321.37498843 25321.37523148]


In [16]:
# Reading the Forecasted the Dataset
# The forecasted dataset can be downloaded by applying the filters on the website.
#The feature used for comparison of the 
import netCDF4
from netCDF4 import Dataset
import numpy as np
file1 = Dataset("dataset-bal-analysis-forecast-phy-dailymeans_1556710689517.nc",'r')
print(file1.variables)
ba=file1.variables["depth"][:]
da=file1.variables["lat"][:]
ea=file1.variables["thetao"][:]
fa=file1.variables["lon"][:]
i=0
j=0
k=0
aw=[]
bc=[]
cd=[]
while(i<len(ea)):
    b=np.ma.mean(ea[i])
    if (b=="nan"):
        aw.append("nan")
    else:    
        aw.append(b)
    i+=1
print((len(aw))) 
po=file1.variables["time"][:]
print(po)

OrderedDict([('depth', <class 'netCDF4._netCDF4.Variable'>
float32 depth(depth)
    standard_name: depth
    long_name: depth
    units: m
    positive: down
    axis: Z
    unit_long: meters
    _CoordinateAxisType: Height
    _CoordinateZisPositive: down
    valid_min: 0.0
    valid_max: 5.0
unlimited dimensions: 
current shape = (2,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('sob', <class 'netCDF4._netCDF4.Variable'>
float32 sob(time, lat, lon)
    standard_name: sea_water_salinity
    long_name: Sea water salinity at sea floor
    units: 0.001
    _FillValue: -999.0
    missing_value: -999.0
    unit_long: practical salinity unit
    _ChunkSizes: [  1 773 767]
unlimited dimensions: 
current shape = (2, 38, 3)
filling on), ('vo', <class 'netCDF4._netCDF4.Variable'>
float32 vo(time, depth, lat, lon)
    long_name: Northward current
    units: m s-1
    _FillValue: -999.0
    missing_value: -999.0
    unit_long: meters per second
    standard_name: northward_sea

In [9]:
# This code is for the conversion to the list format for better interpretation.  
aw=[]
aa=[]
ab=[]
ac=[]
ad=[]
m=0
while(m<len(po)):
    k=0
    while(k<len(ba)):
        j=0
        while(j<len(da)):
            i=0
            while(i<len(fa)):
                aw.append(ea[m,k,j,i])
                aa.append(fa[i])
                ab.append(da[j])
                ac.append(ba[k])
                ad.append(po[m])
                i+=1
            j+=1 
        k+=1 
    m+=1

In [10]:
# For writing into csv format.
import pandas as pd
df_result=pd.DataFrame(list(zip(ad,ac,ab,aa,aw)),columns=['Time','Depth',"Latitude","Longitude","Temperature"])
df_result.to_csv('Testtemp'+('.csv'))

In [20]:
import numpy as np
import pandas as pd
fil=pd.read_csv("TempGreifswalderoi.csv")
Temp=fil['Temperature']
lati=fil['Latitude']
long=fil['Longitude']
Date=fil['Time']
aw=[]
ty=set(lati)
tu=set(long)
i=0
uy=[]
aw=[]
am=[]
at=[]
at.append
while(i<(len(lati)-1)):
    if lati[i]==lati[i+1]:
        j=0
        while(j<(len(long)-1)):
            if long[j]==long[j+1]:
                at.append(Temp[j])
                aw.append(long[j])
                am.append(lati[j])
            else:    
                at.append('nan')
                aw.append('nan')
                am.append('nan')
            j+=1
    i+=1    
t=0
er=[0]
while(t<(len(at))):
    if aw[t]=='nan':
        er.append(t)
    t+=1
r=0
pl=[]
oi=[]
po=[]
t=0
while(t<(len(er)-1)):
    qw=at[(er[t]):(er[t+1])]
    ty=am[(er[t]):(er[t+1])]
    ui=aw[(er[t]):(er[t+1])]
    rt=0
    il=[]
    y=0
    while(rt<len(qw)):
        if qw[rt]=='nan':
            il.append(rt)
        rt+=1
    
    while(y<len(il)):
        qw.pop(il[y]) 
        y+=1
    pl.append(qw[0])
    y=0
    while(rt<len(ty)):
        if ty[rt]=='nan':
            il.append(rt)
        rt+=1
    
    while(y<len(il)):
        ty.pop(il[y]) 
        y+=1
    po.append(ty[0])
    y=0
    while(rt<len(ui)):
        if ui[rt]=='nan':
            il.append(rt)
        rt+=1
    
    while(y<len(il)):
        ui.pop(il[y]) 
        y+=1
    oi.append(ui[0]) 
    t+=1
print(pl)
print(po)
print(oi)
#We need to convert our oi to list format.
#We load different dates datasets and find out the number of different values.

    
    
    
    
    
    
        
        
    

['[11.4]', '7.2', '7.2', '7.2', '7.2', '7.2', '7.2', '7.2', '7.2', '7.2', '7.2', '7.2', '7.2', '7.2', '7.2', '7.2', '7.2', '7.2', '7.2', '7.2', '7.2']
[54.25, 54.88330078, 54.88330078, 54.88330078, 54.88330078, 54.88330078, 54.88330078, 54.88330078, 54.88330078, 54.88330078, 54.88330078, 54.88330078, 54.88330078, 54.88330078, 54.88330078, 54.88330078, 54.88330078, 54.88330078, 54.88330078, 54.88330078, 54.88330078]
[13.92000008, 13.86670017, 13.86670017, 13.86670017, 13.86670017, 13.86670017, 13.86670017, 13.86670017, 13.86670017, 13.86670017, 13.86670017, 13.86670017, 13.86670017, 13.86670017, 13.86670017, 13.86670017, 13.86670017, 13.86670017, 13.86670017, 13.86670017, 13.86670017]


In [30]:
test_list = pl
res = max(set(test_list), key = test_list.count)
print(res)
#We can find out the value of temperature of maximum frequency of a particular day.




7.2


## Conversion to SQL Database 

In [24]:
#This can be used for the conversion of csv files to sqlite Database
from __future__ import print_function
import sqlite3
import csv
import os
import glob
import sys
 
db = sys.argv[1]
 
conn = sqlite3.connect(db)
conn.text_factory = str  # allows utf-8 data to be stored
 
c = conn.cursor()
 
# traverse the directory and process each .csv file
for csvfile in glob.glob(os.path.join(sys.argv[2], "*.csv")):
    # remove the path and extension and use what's left as a table name
    tablename = os.path.splitext(os.path.basename(csvfile))[0]
 
    with open(csvfile, "rb") as f:
        reader = csv.reader(f)
 
        header = True
        for row in reader:
            if header:
                # gather column names from the first row of the csv
                header = False
 
                sql = "DROP TABLE IF EXISTS %s" % tablename
                c.execute(sql)
                sql = "CREATE TABLE %s (%s)" % (tablename,
                          ", ".join([ "%s text" % column for column in row ]))
                c.execute(sql)
 
                for column in row:
                    if column.lower().endswith("_id"):
                        index = "%s__%s" % ( tablename, column )
                        sql = "CREATE INDEX %s on %s (%s)" % ( index, tablename, column )
                        c.execute(sql)
 
                insertsql = "INSERT INTO %s VALUES (%s)" % (tablename,
                            ", ".join([ "?" for column in row ]))
 
                rowlen = len(row)
            else:
                # skip lines that don't have the right number of columns
                if len(row) == rowlen:
                    c.execute(insertsql, row)
 
        conn.commit()
 
c.close()
conn.close()

## Mean Squared Errors 

In [31]:
#Computation for the mean_squared_errors between forecasted dataset and the actual insitu observations.
#Computation of the lists can be found out for different days as described in the above procedure.
#l1=list for forecasted weather data
#l2=list for actual insitu observations
#The data is for two days.
l1=[7.2,11.15]
l2=[7.736523,9.96]
from sklearn.metrics import mean_squared_error
aw=mean_squared_error(l1,l2)
print(aw)


0.8519784647644993


* Similarly, we can do the analysis for different parameters and at different depths.